In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, ElementNotInteractableException, NoSuchElementException
import re
import pandas as pd
import time 




In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(chrome_options=options)
driver.get("http://3.218.228.126/vadu.dll")

In [53]:
USUARIO = "daniel@sthima.com.br"
SENHA = "Score@123"
TEMPO_ATT_PROTESTO = 3


In [100]:
def make_login(driver):
    user_input = driver.find_element_by_name('Codigo')
    key_input = driver.find_element_by_name('Senha')
    
    user_input.clear()
    user_input.send_keys(USUARIO)
    
    key_input.clear()
    key_input.send_keys(SENHA)
    
    driver.find_element_by_css_selector("button.btn").click()

def login(driver):
    try:
        driver.get("http://3.218.228.126/vadu.dll")
        driver.find_element_by_xpath('//*[@id="barra-icones"]/li[6]/a').click()
    except:
        pass
        
        
    driver.get("http://3.218.228.126/vadu.dll")
    if len(driver.find_elements_by_class_name('g-recaptcha-outer')):
        driver.close()
        
        aux_driver = webdriver.Firefox()
        aux_driver.get("http://3.218.228.126/vadu.dll")
        make_login(aux_driver)
        close_modal(aux_driver)
        aux_driver.close()
        
        driver = webdriver.Chrome()
        driver.get("http://3.218.228.126/vadu.dll")
    
    make_login(driver)
    
    return driver

In [102]:
driver = login(driver)

In [56]:
def close_modal(driver):
    close_button = driver.find_elements_by_id("naoRedirecionaInfoCredit")
    if len(close_button):
        try:
            close_button[0].click()
        except:
            pass

In [103]:
close_modal(driver)

In [58]:


def search_cnpj(driver, cnpj):
    cnpj_input = driver.find_element_by_name('q')
    try:
        cnpj_input.clear()
    except:
        pass
    cnpj_input.send_keys(cnpj)
    cnpj_input.send_keys(Keys.ENTER)
    


In [59]:
def necessary_update(driver):
    last_update = driver.find_element_by_xpath('//*[@id="tabProtestos"]/div/div[4]/div/div[5]/h5').text.strip()
    
    def separet_number_and_letters(s):
        numbers = []
        text = []

        for i in s:
            try:
                float(i)
                numbers.append(i)
            except:
                text.append(i)
                
        try:
            number = float(''.join(numbers))
        except:
            number = 0

        return number, ''.join(text)
    
    time1, time2 = separet_number_and_letters(last_update)
    
    if 'mes' in time2 and time1 >= TEMPO_ATT_PROTESTO:
        return True
    elif 'ano' in time2:
        return True
    elif 'não consultado' in time2:
        return True
    else:
        return False



In [120]:
def wait_load(driver, delay = 60):
    try:
        start = time.time()
        while driver.find_element_by_class_name('valor-total-protestos').text == '' or (time.time() - start) > delay:
            driver.find_element_by_xpath('//a[@href="#tabProtestos"]').click()
            if driver.find_element_by_class_name('totalProtestos').text == '?':
                break

        return True
    except TimeoutException:
        return False
    
def wait_load_button(driver, delay = 60):
    try:
        element = WebDriverWait(driver,delay).until(EC.element_to_be_clickable((By.ID, "btnAcaoAtualizarProtesto")))
        return True
    except TimeoutException:
        return False

def DEFAULT_OBJ(cnpj):
        return {'cnpj':cnpj,
            'faturamento':np.nan,
            'funcionarios':np.nan,
            'total_protesto':np.nan,
            'valor_protesto':np.nan} 
    
def capture_VADU_info(driver, cnpj):
    driver.get("http://3.218.228.126/vadu.dll")
    driver = login(driver)
    close_modal(driver)

    search_cnpj(driver, cnpj)
    result_search = driver.find_elements_by_id('ResultadoBusca')
    
    if len(result_search):
        if result_search.text == 'Pesquisa sem resultados':
            return DEFAULT_OBJ(cnpj)
        
    cnpj = driver.find_element_by_id('cnpjBlocoDinamico').text
    faturamento = driver.find_element_by_id('valorFaturamentoEstimado').text
    funcionarios = driver.find_element_by_id('valorQuantidadeFuncionarios').text
    
    protest_button = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, '//a[@href="#tabProtestos"]')))
    protest_button.click()
    if not(wait_load(driver)):
            return DEFAULT_OBJ(cnpj)
    
    if necessary_update(driver):
        driver.find_element_by_id('btnAcaoAtualizarProtesto').click()
        if not(wait_load_button(driver)):
            return DEFAULT_OBJ(cnpj)
    
    total_protesto = driver.find_element_by_class_name('totalProtestos').text
    valor_protesto = driver.find_element_by_class_name('valor-total-protestos').text
    driver.close()
    
    return {'cnpj':cnpj,
            'faturamento':faturamento,
            'funcionarios':funcionarios,
            'total_protesto':total_protesto,
            'valor_protesto':valor_protesto}

In [41]:
import pymongo
import pandas as pd
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["ATFCredit"]
mycol = mydb['Txt_features']
cursor = mycol.find({'result': {'$ne': -1}})
df = pd.DataFrame(list(cursor))

In [121]:

def fill_info_vector(driver, cnpj_vector, i, vadu_infos, n_try = 0 ):
    if i >= len(cnpj_vector):
        return vadu_infos, i
    else:
        driver = webdriver.Chrome(chrome_options=options)
        try:
            vadu_infos.append(capture_VADU_info(driver,cnpj_vector[i]))
            return fill_info_vector(driver, cnpj_vector, i+1, vadu_infos)
        except Exception as e:
            try:
                driver.close()
            except:
                pass
            n_try += 1
            if n_try >= 3:
                vadu_infos.append(cnpj_vector[i])
                return fill_info_vector(driver, cnpj_vector, i+1, vadu_infos)
            else:
                return fill_info_vector(driver, cnpj_vector, i, vadu_infos, n_try)
    

In [123]:
vadu_infos = []
result, ind = fill_info_vector(driver, df['cnpj'], 70, [])

<ipython-input-121-abf577c39365>:19: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [133]:
df_final = pd.DataFrame() 
for i in result:
    df_final = df_final.append(pd.DataFrame([i]), ignore_index = True)




In [145]:
df_final = df_final.drop(index = df_final[df_final[0].notnull()].index, columns = [0]).reset_index(drop = True)

In [147]:

df_final.to_csv('FIM.csv')

In [15]:
result[0]

[{'cnpj': '01.314.776/0001-49',
  'faturamento': 'de 1.500.000 até 4.800.000',
  'funcionarios': 'de 10 até 50',
  'total_protesto': '36',
  'valor_protesto': '9.196.456,58'},
 {'cnpj': '00.061.315/0001-49',
  'faturamento': 'de 100.000.000 até 300.000.000',
  'funcionarios': 'de 50 até 100',
  'total_protesto': '2',
  'valor_protesto': '64.680,62'},
 {'cnpj': '00.863.529/0001-39',
  'faturamento': 'de 10.000.000 até 30.000.000',
  'funcionarios': 'de 50 até 100',
  'total_protesto': '0',
  'valor_protesto': '-'},
 {'cnpj': '83.109.579/0001-52',
  'faturamento': 'de 10.000.000 até 30.000.000',
  'funcionarios': 'de 100 até 500',
  'total_protesto': '21',
  'valor_protesto': '28.991.636,43'},
 {'cnpj': '88.371.075/0001-20',
  'faturamento': 'de 1.500.000 até 4.800.000',
  'funcionarios': 'de 10 até 50',
  'total_protesto': '0',
  'valor_protesto': '-'},
 {'cnpj': '86.837.366/0001-35',
  'faturamento': 'de 30.000.000 até 100.000.000',
  'funcionarios': 'de 100 até 500',
  'total_protesto

In [5]:
df1 = pd.read_csv('FIM.csv',index_col = 0)
df2 = pd.read_csv('0-70_cnpjs.csv',index_col = 0)

In [9]:
df = pd.concat([df1,df2])

In [11]:
df.to_csv('Base_Crawler.csv')